**Library Imports**

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

**Reading Labels file to get the unique breeds in the dataset**

In [2]:
labels_csv = pd.read_csv("labels.csv")

In [3]:
unique_breeds = np.unique(labels_csv.breed)

**Functions to convert image to tensor**

In [4]:
IMAGE_SIZE = 224

def image_to_tensor(imagepath):
  #Read file
  image = tf.io.read_file(imagepath)
  #Convert file in a numerical tensor
  image = tf.image.decode_jpeg(image, channels=3)
  #Convert values between 0-255 to 0-1
  image = tf.image.convert_image_dtype(image, tf.float32)
  #Resize the image
  image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])

  return image


In [5]:
def get_image_label(path_image, label):
  image = image_to_tensor(imagepath=path_image)
  return image, label

In [6]:
BATCH_SIZE = 32

def create_data_batches(X, y=None, batch_size = BATCH_SIZE, valid_data= False, test_data= False):
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X)))

    data_batch = data.map(image_to_tensor).batch(BATCH_SIZE)
    return data_batch

  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))

    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

  else:
    print("Creating training data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
    #Shuffle
    data = data.shuffle(buffer_size=len(X))

    data = data.map(get_image_label)

    data_batch = data.batch(BATCH_SIZE)
    return data_batch


**Function to load the saved trained model**

In [7]:
def load_model(model_path):
  print(f"Loading a model from {model_path}")
  model = tf.keras.models.load_model(model_path, custom_objects={"KerasLayer": hub.KerasLayer})
  return model

**Loading the model**

In [9]:
model = load_model("model.h5")

Loading a model from model.h5


TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

**Predicting the breed**

In [ ]:
def predict_image(image_path):
    image_batch = create_data_batches(image_path, test_data=True)

    prediction = model.predict(image_batch, verbose=True)

    index = 0
    img = mpimg.imread(image_path[index])
    imgplot = plt.imshow(img)
    
    ax = plt.gca()

    ax.get_xaxis().set_visible(False)

    ax.get_yaxis().set_visible(False)
    
    plt.show()

    print(f"Max value:  {(np.max(prediction[index])) * 100}")
    print(f"Predicted Breed:  {unique_breeds[np.argmax(prediction[index])]}")


In [ ]:
predict_image(["dog/tofa_5.jpg"])